# Singapore Attraction Discovery Agent
Agentic system for discovering Singapore attractions using tool-based reasoning, LLM decision-making, and iterative refinement to meet user requirements.

In [1]:
# Install requirements if needed
# !pip install -r requirements.txt

In [2]:
# Imports
import os
import sys
import json
import logging
from typing import Dict, List, Any, Optional
from dotenv import load_dotenv

# Add src to path
sys.path.append(os.path.join(os.path.dirname('.'), 'src'))

try:
    from openai import OpenAI
except ImportError:
    OpenAI = None

try:
    from anthropic import Anthropic
except ImportError:
    Anthropic = None

In [3]:
# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(),
        logging.FileHandler('agent_reasoning.log', mode='a')
    ]
)
logger = logging.getLogger(__name__)

In [4]:
# Load environment variables
load_dotenv()

# Check API keys
print("API Keys Status:")
print(f"GOOGLE_MAPS_API_KEY: {'✓ Set' if os.getenv('GOOGLE_MAPS_API_KEY') else '✗ Missing'}")
print(f"CLIMATIQ_API_KEY: {'✓ Set' if os.getenv('CLIMATIQ_API_KEY') else '✗ Missing'}")
print(f"OPENAI_API_KEY: {'✓ Set' if os.getenv('OPENAI_API_KEY') else '✗ Missing'}")
print(f"ANTHROPIC_API_KEY: {'✓ Set' if os.getenv('ANTHROPIC_API_KEY') else '✗ Missing'}")

API Keys Status:
GOOGLE_MAPS_API_KEY: ✓ Set
CLIMATIQ_API_KEY: ✓ Set
OPENAI_API_KEY: ✓ Set
ANTHROPIC_API_KEY: ✗ Missing


## Input Data Loading
Load and validate input file with trip requirements.

In [5]:
def load_input_file(file_path: str) -> dict:
    """Load and validate input file."""
    try:
        with open(file_path, 'r') as f:
            input_data = json.load(f)
        
        # Basic validation
        required_fields = ["trip_dates", "duration_days", "budget", "pace"]
        for field in required_fields:
            if field not in input_data:
                raise ValueError(f"Missing required field: {field}")
        
        # Check if optional section exists, and validate accommodation_location if it does
        if "optional" in input_data and "accommodation_location" not in input_data["optional"]:
            raise ValueError("Missing required field: optional.accommodation_location")
        
        return input_data
    
    except FileNotFoundError:
        print(f"Error: Input file '{file_path}' not found.")
        return None
    except json.JSONDecodeError as e:
        print(f"Error: Invalid JSON in input file: {e}")
        return None
    except ValueError as e:
        print(f"Error: {e}")
        return None

# Test with different input files
print("Available input files:")
for file in os.listdir('inputs'):
    if file.endswith('.json'):
        print(f"  inputs/{file}")

Available input files:
  inputs/adventure_input.json
  inputs/complex_input.json
  inputs/family_budget_input.json
  inputs/garden_only_input.json
  inputs/simple_input.json


## Places Research Agent - Focused Implementation
Handles location-based attraction discovery using Google Places API

In [6]:
from tools import search_places, search_multiple_keywords, get_place_details

class PlacesResearchAgent:
    """
    Specialized agent for location-based attraction research using Google Places API.
    """

    def __init__(self, system_prompt=""):
        # Initialize OpenAI client
        self.client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
        
        # Agent configuration
        self.model_name = "gpt-4o"
        self.temperature = 0.3
        
        # Message history
        self.messages = []
        self.system_prompt = system_prompt or self._get_default_system_prompt()
        
        if self.system_prompt:
            self.messages.append({"role": "system", "content": self.system_prompt})
        
        # Tool definitions for Google Places API
        self.available_tools = self._define_places_tools()
        
        # Known actions mapping
        self.known_actions = {
            "search_places": search_places,
            "search_multiple_keywords": search_multiple_keywords,
            "get_place_details": get_place_details
        }

        # Agent state for tracking requirements
        self.required_places = 0
        self.current_results_count = 0
    
    def _get_default_system_prompt(self) -> str:
        """Default system prompt following thought-action-observation pattern."""
        return """
            You are a places and attractions research specialist. You run in a loop of Thought, Action, Observation.
            At the end of the loop you output an Answer.
            
            Use Thought to describe your reasoning about the places and attractions research.
            Use Action to run one of the actions available to you.
            Observation will be the result of running those actions.
            
            INTEREST MAPPING RULES:
            Before processing any interests, map them to the following standardized categories:
            - tourist_attraction
            - food
            - cafe
            - bar
            - bakery
            - park
            - museum
            - shopping_mall
            - lodging
            
            Mapping examples:
            - "parks" to park
            - "museums" to museum
            - "family" to tourist_attraction
            - "educational" to museum or tourist_attraction
            - "gardens" to park
            - "nature" to park
            - "culture" to museum or 'cultural food'
            - "dining" to food
            - "coffee" to cafe
            - "shopping" to shopping_mall
            - "accommodation" to lodging
            
            Special handling for food-related interests:
            If an interest cannot be classified into the above categories but refers to food options 
            (like "vegetarian", "halal", "vegan", "local cuisine"), rewrite it as "[interest] food".
            Examples:
            - "vegetarian" → "vegetarian food"
            - "halal" → "halal food"
            - "local cuisine" → "local food"
            
            Your available actions are:
            1. search_places when there is only ONE interest/keyword to search for.
            2. search_multiple_keywords when there are MULTIPLE interests/keywords to search for.
            3. get_place_details to get comprehensive details about all places.
            
            The 'interests' input from user maps to:
            - 'keyword' parameter for search_places (single string)
            - 'keywords' parameter for search_multiple_keywords (array of strings)

            Finds attractions near accomodation location and find at least one food places per day.
            
            Set ratings filter to a 3.5 for shopping malls.
            Set ratings filter to a 4.2 for food places.
            Set ratings filter to a 4.0 for all other place types.
            
            Focus on providing comprehensive attraction recommendations based on user interests and accommodation location.
            Always consider practical factors like location, ratings, accessibility needs, and user preferences.
            Calculate required places based on pace and duration, and expand search if insufficient results are found.
            """.strip()

    def _define_places_tools(self) -> List[Dict]:
        """Define Google Places API tools."""
        return [
            {
                "type": "function",
                "function": {
                    "name": "search_places",
                    "description": "Search for places with a SINGLE keyword/interest based on user interests and accommodation location",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "object",
                                "properties": {
                                    "lat": {"type": "number", "description": "Latitude of accommodation"},
                                    "lng": {"type": "number", "description": "Longitude of accommodation"},
                                    # "neighborhood": {"type": "string", "description": "Neighborhood name (optional)"}
                                },
                                "required": ["lat", "lng"],
                                "description": "Accommodation location with coordinates and optional neighborhood"
                            },
                            "keyword": {
                                "type": "string",
                                "description": "A single user interest like 'tourist_attraction'"
                            },
                            "radius": {
                                "type": "integer",
                                "description": "Search radius in meters (default: 5000, increase by 2000 if not enough results)",
                                "default": 5000
                            },
                            "max_pages": {
                                "type": "integer",
                                "description": "Maximum number of pages to retrieve (default: 3), Each page has up to 20 results.",
                                "default": 3
                            },
                            "min_rating": {
                                "type": "number",
                                "description": "Minimum rating filter (1.0-5.0)",
                                "default": 4.0
                            }
                        },
                        "required": ["location", "keyword"]
                    }
                }
            },
            {
                "type": "function",
                "function": {
                    "name": "search_multiple_keywords",
                    "description": "Search for places with MULTIPLE keywords/interests based on user interests and accommodation location",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "object",
                                "properties": {
                                    "lat": {"type": "number", "description": "Latitude of accommodation"},
                                    "lng": {"type": "number", "description": "Longitude of accommodation"},
                                    # "neighborhood": {"type": "string", "description": "Neighborhood name (optional)"}
                                },
                                "required": ["lat", "lng"],
                                "description": "Accommodation location with coordinates and optional neighborhood"
                            },
                            "keywords": {
                                "type": "string",
                                "items": {"type": "string"},
                                "description": "A single user interest like 'tourist_attraction'"
                            },
                            "radius": {
                                "type": "integer",
                                "description": "Search radius in meters (default: 5000, increases if not enough results)",
                                "default": 5000
                            },
                            "max_pages": {
                                "type": "integer",
                                "description": "Maximum number of pages to retrieve (default: 1), Each page has up to 20 results.",
                                "default": 1
                            },
                            "min_rating": {
                                "type": "number",
                                "description": "Minimum rating filter (1.0-5.0)",
                                "default": 4.0
                            }
                        },
                        "required": ["location", "keywords"]
                    }
                }
            },
            {
                "type": "function",
                "function": {
                    "name": "get_place_details",
                    "description": "Get comprehensive detailed information about on a list of places",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "place_id": {
                                "type": "array",
                                "items": {"type": "string"},
                                "description": "Google Places API place_id"
                            },
                            "fields": {
                                "type": "array",
                                "items": {"type": "string"},
                                "description": "Specific fields to retrieve",
                                "default": ["name", "formatted_address", "geometry", "opening_hours", "rating", "website", "price_level", "type"]
                            }
                        },
                        "required": ["place_id"]
                    }
                }
            }
        ]

    def __call__(self, message: str) -> str:
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result
    
    def execute(self) -> str:
        """Execute with function calling support."""
        response = self.client.chat.completions.create(
            model=self.model_name,
            temperature=self.temperature,
            messages=self.messages,
            tools=self.available_tools,
            tool_choice="auto"
        )
        
        message = response.choices[0].message
        
        # Handle function calls if present
        if message.tool_calls:
            # Add assistant message with tool calls
            self.messages.append({
                "role": "assistant",
                "content": message.content,
                "tool_calls": message.tool_calls
            })
            
            # Execute tool calls
            for tool_call in message.tool_calls:
                tool_result = self._execute_tool_call(tool_call)
                
                # Add tool result to messages
                self.messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "content": json.dumps(tool_result, default=str)
                })
            
            # Get final response after tool execution
            final_response = self.client.chat.completions.create(
                model=self.model_name,
                temperature=self.temperature,
                messages=self.messages
            )
            
            return final_response.choices[0].message.content
        
        return message.content
    
    def _execute_tool_call(self, tool_call) -> Dict[str, Any]:
        """Execute tool call and return results."""
        tool_name = tool_call.function.name
        tool_args = json.loads(tool_call.function.arguments)
        
        print(f"RESEARCH AGENT: Executing {tool_name} with args: {tool_args}")
        
        if tool_name in self.known_actions:
            return self.known_actions[tool_name](**tool_args)
        else:
            return {"error": f"Unknown tool: {tool_name}"}

    # =====================================
    # AGENT INTERNAL LOGIC
    # =====================================

    def calculate_required_places(self, pace: str, duration_days: int) -> int:
        """
        Agent's internal goal-setting: Calculate required number of places.
        This sets the target for the agent to achieve through tool usage.
        No artificial limits - agent will expand search radius to meet the target.
        """
        try:
            pace_mapping = {
                "slow": 2,
                "relaxed": 4,
                "moderate": 4,
                "active": 6,
                "standard": 6,
                "fast": 8,
                "intensive": 8
            }
            
            multiplier = float(os.getenv("MAX_PLACES_MULTIPLIER", "2"))
            pace_value = pace_mapping.get(pace.lower(), 6)
            required_places = int(pace_value * duration_days * multiplier)
            
            # Only enforce minimum, no maximum cap
            required_places = max(required_places, 15)  # Minimum 15 places

            print(f"RESEARCH AGENT: Goal set - {required_places} places for {pace} pace over {duration_days} days")
            return required_places
            
        except (ValueError, TypeError, AttributeError):
            print("RESEARCH AGENT: Failed to calculate required places, using default: 15")
            return 15

In [7]:
def run_agentic_workflow(self, input_data):
    """Run agent workflow: calculate required places and fetch attractions."""
    # Extract parameters
    interests = input_data.get('optional', {}).get('interests', [])
    accommodation_location = input_data.get('optional', {}).get('accommodation_location', {})
    pace = input_data.get('pace', 'moderate')
    duration_days = input_data.get('duration_days', 1)
    radius = 20000
    min_rating = 4.0
    
    # Calculate required places
    required_places = self.calculate_required_places(pace, duration_days)
    print(f"Agent will fetch at least {required_places} attractions.")
    
    # Fetch places using search_places tool
    results = self.known_actions['search_places'](interests, accommodation_location, radius, min_rating)
    
    # Truncate to required_places if needed
    if len(results) > required_places:
        results = results[:required_places]
    
    print(f"Found {len(results)} places:")
    for place in results:
        print(f"{place.get('name', 'Unknown')} (Rating: {place.get('rating', 'N/A')})")
    return results

In [8]:
# Load input data
input_file = 'inputs/garden_only_input.json'
input_data = load_input_file(input_file)

response = run_agentic_workflow(PlacesResearchAgent(), input_data)

print(response)

2025-09-27 00:44:41,700 - googlemaps.client - INFO - API queries_quota: 60


RESEARCH AGENT: Goal set - 16 places for relaxed pace over 2 days
Agent will fetch at least 16 attractions.


ValueError: location must be (lat, lng) tuple or dict with 'lat'/'lng' keys